In [2]:
import pandas as pd
import glob
import xarray as xr
import numpy as np
from statsmodels.stats.anova import AnovaRM 
from scipy import stats
from matplotlib import pyplot as plt
import pingouin as pg
import numpy as np

In [ ]:
#APPROACH 1 

In [65]:
vn = ['fusiform-rh', 'fusiform-lh', 'lingual-lh', 'lingual-rh',
'cuneus-rh','cuneus-lh', 'lateraloccipital-rh', 'lateraloccipital-lh']

methods = ['coh','ciplv','imcoh','wpli2']

In [67]:
#intra-subject variance

intra_variance_dict = {}
for method in methods: 
    subject_files = glob.glob(f'/Users/lina_01/Desktop/mpi_lemon/undirected_outputs/{method}/*_EC.nc')
    variance_per_subject = []
    for file in subject_files:
        xarray = xr.open_dataarray(file)
        std_mean_list = []
        for sample in range(0,100):
            std = xarray.sel(bootstrap_samples=sample, region1=vn, region2=vn).values.std()
            mean = xarray.sel(bootstrap_samples=sample, region1=vn, region2=vn).values.mean()
            std_mean_list.append(mean)
        variance_per_subject.append(np.std(std_mean_list)**2)
    intra_variance_dict[method] = variance_per_subject

In [69]:
#Inter-subject variance
inter_variance_dict = {}
for method in methods:
    subject_files = glob.glob(f'/Users/lina_01/Desktop/mpi_lemon/undirected_outputs/{method}/*_EC.nc')
    all_bootstraps = []  #list of 11200 arrays
    for file in subject_files:
        xarray = xr.open_dataarray(file)
        for sample in range(0,100):
            bootstrap = xarray.sel(bootstrap_samples=sample, region1=vn, region2=vn).values
            all_bootstraps.append(bootstrap)
    
    random_bootstraps = []
    for i in range(0,112):
        index = np.random.choice(range(0,11200),100)
        sample = np.array(all_bootstraps)[index,:]
        random_bootstraps.append(sample)

    variance_per_bootstrap = []
    for sample in random_bootstraps:
        variance = sample.std()
        variance_per_bootstrap.append(variance)

    inter_variance_dict[method] = variance_per_bootstrap

In [80]:
p_vals = {}
eff_size = {}
for method in methods:
    p_val = stats.ranksums(inter_variance_dict[method], intra_variance_dict[method])[1]
    
    n1 = len(inter_variance_dict[method])
    n2 = len(intra_variance_dict[method])
    sd1 = np.std(inter_variance_dict[method])
    sd2 = np.std(intra_variance_dict[method])

    mean1 = np.mean(inter_variance_dict[method])
    mean2 = np.mean(intra_variance_dict[method])

    pooled_sd = np.sqrt((((n1-1)*(sd1**2))+((n2-1)*(sd2**2)))/(n1+n2-2))

    cohens_d = (mean1 - mean2) / pooled_sd

    p_vals[method] = p_val
    eff_size[method] = cohens_d

    

